PIPELINE DE EXTRACCIÓN DE DATOS

In [71]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import time 
import pandas as pd
import random, string
from selenium.webdriver.common.by import By

In [19]:
driver = webdriver.Chrome(executable_path='/home/florian/inteligencia-artificial/publicar-investigacion/nlp-as-comentarios-restaurantes-unmsm/chromedriver')
#driver.get("https://www.google.com.pe/maps/search/cevicheria/@-12.0626704,-77.029618,13z/data=!3m1!4b1?hl=es-419");

/tmp/ipykernel_72409/2206980964.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/home/florian/inteligencia-artificial/publicar-investigacion/nlp-as-comentarios-restaurantes-unmsm/chromedriver')


Obtener la estructura de la pagina web página incial

In [105]:
def saveFile(title, cordenate, stars, comments):
    frame = pd.DataFrame({'titulo' : title,'coordenada' : cordenate,'stars' : stars, 'comment' : comments})
    file_name = title+"-"+str(random.randint(10000, 100000))+".csv"
    frame.to_csv(file_name, index = None, header=True,encoding='utf-8')
    

In [21]:
def scrollDown(elemento, numberOfScrollDowns):
    while numberOfScrollDowns >=0:
        elemento.send_keys(Keys.PAGE_DOWN)
        numberOfScrollDowns -= 1
        time.sleep(3)
    #return browser

PROCESO DE SCRAPING

In [154]:
driver.get("https://www.google.com/maps/search/restaurant/@-12.1187799,-77.0235511,17z/data=!4m4!2m3!5m1!4e1!6e5");

In [156]:
#inputElement = driver.find_element_by_id("searchboxinput")
#inputElement.send_keys('piso 21')
page_header = BeautifulSoup(driver.page_source)
list_restaurant = page_header.find_all('div', { 'class': 'Nv2PK THOPZb CpccDe' })
for restaurant in list_restaurant:
    try:
        time.sleep(10)
        #page_header = BeautifulSoup(driver.page_source)
        title = restaurant.find("div", {"class": "qBF1Pd fontHeadlineSmall"}).text
        quantity = restaurant.find("span", { "class": "UY7F9" }).text
        quantity_reviews = filterQuantity(quantity)
        
        if(quantity_reviews > 4000):
            elem = int(quantity_reviews / 3) - 500
        elif quantity_reviews > 1000:
            elem = int(quantity_reviews / 3) - 100
        else:
            elem = int(quantity_reviews / 3)
    
        elemento = driver.find_elements(By.CLASS_NAME,'Nv2PK')
        elemento = elemento[0]
        elemento.click()
        time.sleep(10)
        bottonClick = driver.find_elements(By.XPATH, "//button[@class='hh2c6 ']")
        bottonClick = bottonClick[0]
        bottonClick.click()
        time.sleep(10)
        #print(BeautifulSoup(driver.page_source))
        elementScroll = driver.find_elements(By.XPATH, "//div[@class='m6QErb DxyBCb kA9KIf dS8AEf ']")
        #print(len(elementScroll), "##########################")
        scrollDown(elementScroll[0], 5)
        
        currentURL = driver.current_url;
        cordenate = currentURL[currentURL.find('@')+1:currentURL.find('z')+1]
        
        page_reviews = BeautifulSoup(driver.page_source)
        comments = [item.text for item in page_reviews.find_all('span', {'class' : 'wiI7pd'})]
        stars = [len(item.find_all('img', {'class' : 'hCCjke vzX5Ic'})) for item in page_reviews.find_all('span', {'class' : 'kvMYJc'})]
        saveFile(title, cordenate, stars, comments)
        
    except OSError as err:
        print("error ->", err)
        break
    except NoSuchElementException as err:
        print("error ->", err)
        break
    except ElementNotVisibleException as err:
        ##exception for page empty
        print("error ->", err)
        break

3 ####################


In [107]:
import re
def filterQuantity(quantity):
    pattern = r'[(,)]'
    return int(re.sub(pattern,"", quantity)) 